In [1]:
import sapien.core as sapien
from Tools import misc, ForwardKinematics, ModelSim, NumForwardDynamicsDer
import numpy as onp
import jax.numpy as np
from jax import jit, jacfwd, jacrev, grad
from ilqr import ILQR
from datetime import datetime
from tqdm.notebook import trange
import matplotlib.pyplot as plt

%matplotlib notebook
%env XLA_PYTHON_CLIENT_MEM_FRACTION=.40


sim = sapien.Engine()
renderer = sapien.OptifuserRenderer()
sim.set_renderer(renderer)
render_controller = sapien.OptifuserController(renderer)

Using default glsl path /home/zack/anaconda3/envs/ml/lib/python3.7/site-packages/sapien/glsl_shader/130
env: XLA_PYTHON_CLIENT_MEM_FRACTION=.40


In [2]:
DEBUG = False

stabled = False


def create_scene(timestep, visual):
    s = sim.create_scene([0, 0, 0])
    s.add_ground(-1)
    s.set_timestep(timestep)

    loader = s.create_urdf_loader()
    loader.fix_root_link = True
    if visual:
        loader.collision_is_visual = True
        s.set_ambient_light([0.5, 0.5, 0.5])
        s.set_shadow_light([0, 1, -1], [0.5, 0.5, 0.5])

    # build
    robot = loader.load("../../../assets/Arm/panda.urdf")

    for joint in robot.get_joints():
        joint.set_drive_property(stiffness=0, damping=50)
    robot.set_qpos(np.array([0, 1.7, 0, -1.5, 0, 2.5, 0.7, 0.4, 0.4]))

    return s, robot


sim_timestep = 1 / 60
optim_timestep = 1 / 60
s0, robot = create_scene(sim_timestep, True)

render_controller.set_camera_position(-5, 0, 0)
render_controller.set_current_scene(s0)


In [4]:
robot.compute_mass_matrix().shape

(9, 9)

In [3]:
u_range = robot.get_qlimits().T
factor = 4
adjusted_u_range = u_range * factor

robot.set_qlimits(adjusted_u_range.T)

adjusted_u_mean = np.mean(adjusted_u_range, axis=0)
adjusted_u_diff = adjusted_u_range[1] - adjusted_u_range[0]

print(robot.get_qlimits())
render_controller.show_window()
while not render_controller.should_quit:
    u = onp.random.random(robot.dof) * adjusted_u_diff - adjusted_u_diff/2 + adjusted_u_mean
    u = np.clip(u, adjusted_u_range[0], adjusted_u_range[1])
    robot.set_qf(u)
    s0.step()
    s0.update_render()
    render_controller.render()

MemoryError: std::bad_alloc